https://opendata.cwb.gov.tw/dataset/forecast/F-D0047-001

In [1]:
import io
import os
import time
import datetime
import json
import requests
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize

In [2]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
from os.path import basename
from email.mime.application import MIMEApplication
from email.utils import COMMASPACE, formatdate

登入帳號

In [3]:
#編輯郵件內容
def send_mail_info(folder):
    content = MIMEMultipart()
    content["subject"] = "file" #郵件主題 
    content["from"] = "m11023032@gemail.yuntech.edu.tw" #自己的郵件地址 
    content["to"] = "m11023032@gemail.yuntech.edu.tw" #傳送到哪裡  
    content.attach(MIMEText(f"the file `{folder}`"))
    send_mail(content)   
#傳送郵件
def send_mail(content):
    with smtplib.SMTP(host="smtp.gmail.com", port="587") as smtp:
        try:
            smtp.ehlo() #確認伺服器是否有回應
            smtp.starttls()#TLS郵件加密模式
            smtp.login("m11023032@gemail.yuntech.edu.tw", "aa0932684719")
            smtp.send_message(content)
            print("Complete!")
        except Exception as e:
            print("Error message: ", e)

In [4]:
folder = 'CWB.3H'

In [5]:
getTime = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print(getTime)
# time.sleep(3600)

2023-03-01 00:22:57


In [6]:
statIDs = ['F-D0047-027']

In [7]:
# # 如果目標資料夾不存在，則新建該資料夾
# for name in statIDs:
#     if not os.path.exists(f'{folder}/{name}'):
#         os.mkdir(f'{folder}/{name}')

# 在每小時02分之前爬取一次

In [8]:

localtime = time.localtime()
result = time.strftime("%M:%S", localtime)
if(result<='60:00'):
    start_time = time.time()
    getTime = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S').replace(':', '%3A')
    authCode = 'CWB-E89A9E94-16F6-4EBE-A690-3950827C2C11'
    #print(getTime)
    for statID in statIDs:
        try:
            url = 'https://opendata.cwb.gov.tw/api/v1/rest/datastore/{}?Authorization={}'.format(statID, authCode)

            header = {}

            r = requests.get(url, headers=header)
            res = r.text
            res_1 = json.loads(res)
            print(res_1)

#                 with io.open('{}/{}/{}.txt'.format(folder, statID, getTime), 'w') as outfile:
#         #             print('save {}/{}/{}.txt'.format(folder, statID, getTime))
#                     outfile.write(res)
#                 time.sleep(0.01)
        except:
            send_mail_info('err: '+folder)
            pass

{'success': 'true', 'result': {'resource_id': 'F-D0047-027', 'fields': [{'id': 'contentDescription', 'type': 'String'}, {'id': 'datasetDescription', 'type': 'String'}, {'id': 'locationsName', 'type': 'String'}, {'id': 'dataid', 'type': 'String'}, {'id': 'locationName', 'type': 'String'}, {'id': 'geocode', 'type': 'Double'}, {'id': 'lat', 'type': 'Double'}, {'id': 'lon', 'type': 'Double'}, {'id': 'elementName', 'type': 'String'}, {'id': 'description', 'type': 'String'}, {'id': 'startTime', 'type': 'Timestamp'}, {'id': 'endTime', 'type': 'Timestamp'}, {'id': 'dataTime', 'type': 'Timestamp'}, {'id': 'value', 'type': 'String'}, {'id': 'measures', 'type': 'String'}]}, 'records': {'locations': [{'datasetDescription': '臺灣各縣市鄉鎮未來1週逐12小時天氣預報', 'locationsName': '雲林縣', 'dataid': 'D0047-027', 'location': [{'locationName': '斗六市', 'geocode': '10009010', 'lat': '23.698834', 'lon': '120.518874', 'weatherElement': [{'elementName': 'PoP12h', 'description': '12小時降雨機率', 'time': [{'startTime': '2023-03-01 

In [9]:
# 基本上最低溫會發生在每天的0~6點，因此最低溫的情況會抓取每天的0~6點時間的值
from datetime import datetime, timedelta

low_temp=res_1['records']['locations'][0]['location'][0]['weatherElement'][8]['time']
max_temp=res_1['records']['locations'][0]['location'][0]['weatherElement'][12]['time']
# total_time = res_1['records']['locations'][0]['location'][0]['weatherElement'][10]
time_format = "%Y-%m-%d %H:%M:%S"
today = datetime.today() 
# 儲存爬蟲的結果
low_answer = pd.DataFrame()
high_answer = pd.DataFrame()
answer = pd.DataFrame()

count_low = 0
count_high = 0


for i in range(len(low_temp)):
    temp_start = datetime.strptime(low_temp[i]['startTime'], time_format)
    temp_end = datetime.strptime(low_temp[i]['endTime'], time_format)
   
#     print(day_low)
#     只抓取每天6~18點的資料
    if (temp_start > today) & (temp_end.hour == 6):
        day_low = int(low_temp[i]['elementValue'][0]['value'])
        value = {"日期":temp_end.date(),'年':temp_end.year,'週':temp_end.isocalendar()[1],"最低溫":day_low,}
        low_answer = low_answer.append(value,ignore_index=True)
    if (temp_start > today) & (temp_start.hour == 6):
        day_high = int(max_temp[i]['elementValue'][0]['value'])
        value = {"日期":temp_end.date(),'年':temp_start.year,'週':temp_start.isocalendar()[1],"最高溫":day_high,}
        high_answer = high_answer.append(value,ignore_index=True)

# temp_start = datetime.strptime(low_temp[1]['startTime'], time_format)
# temp_end = datetime.strptime(low_temp[1]['endTime'], time_format)
# print((temp_start.date() == temp_end.date() and abs(temp_start - temp_end) <= timedelta(hours=24)))
print(low_answer)
print(high_answer)
answer = pd.merge(high_answer,low_answer, on=['日期','年','週'])
for i in range(len(answer)):
    target = answer.loc[i:i]
    answer.loc[i:i,"最高溫差"] = target['最高溫'].values[0] - target['最低溫'].values[0]
answer

           日期     年   週  最低溫
0  2023-03-02  2023   9   15
1  2023-03-03  2023   9   13
2  2023-03-04  2023   9   14
3  2023-03-05  2023   9   13
4  2023-03-06  2023  10   13
5  2023-03-07  2023  10   14
6  2023-03-08  2023  10   15
           日期     年   週  最高溫
0  2023-03-01  2023   9   24
1  2023-03-02  2023   9   22
2  2023-03-03  2023   9   22
3  2023-03-04  2023   9   21
4  2023-03-05  2023   9   23
5  2023-03-06  2023  10   23
6  2023-03-07  2023  10   24


C:\Users\IDSL\AppData\Local\Temp\ipykernel_11000\3118165626.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  high_answer = high_answer.append(value,ignore_index=True)
C:\Users\IDSL\AppData\Local\Temp\ipykernel_11000\3118165626.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  low_answer = low_answer.append(value,ignore_index=True)
C:\Users\IDSL\AppData\Local\Temp\ipykernel_11000\3118165626.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  high_answer = high_answer.append(value,ignore_index=True)
C:\Users\IDSL\AppData\Local\Temp\ipykernel_11000\3118165626.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  low_answer = low_ans

,日期,年,週,最高溫,最低溫,最高溫差
0,2023-03-02,2023,9,22,15,7.0
1,2023-03-03,2023,9,22,13,9.0
2,2023-03-04,2023,9,21,14,7.0
3,2023-03-05,2023,9,23,13,10.0
4,2023-03-06,2023,10,23,13,10.0
5,2023-03-07,2023,10,24,14,10.0


In [11]:
answer

,日期,年,週,最高溫,最低溫,最高溫差
0,2023-03-02,2023,9,22,15,7.0
1,2023-03-03,2023,9,22,13,9.0
2,2023-03-04,2023,9,21,14,7.0
3,2023-03-05,2023,9,23,13,10.0
4,2023-03-06,2023,10,23,13,10.0
5,2023-03-07,2023,10,24,14,10.0


In [16]:
import pyodbc
SERVER = 'DESKTOP-OIEIPGU'
DATABASE = 'NCKUH'
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};\
                       SERVER='+SERVER+';\
                       DATABASE='+DATABASE+';\
                       Trusted_Connection=yes;')
cursor = conn.cursor()
for index, row in answer.iterrows():
     cursor.execute("INSERT INTO weather (date,year,week,max_temperature,min_temperature,diff_temperature) values(?,?,?,?,?,?)", row.日期, row.年	, row.週,row.最高溫,row.最低溫,row.最高溫差)
conn.commit()
cursor.close()

In [13]:
for index, row in answer.iterrows():
    print(row.日期)

2023-03-02
2023-03-03
2023-03-04
2023-03-05
2023-03-06
2023-03-07
